In [5]:
import os
import json
import itertools
import random
import copy
import time
import argparse
from tqdm import tqdm

from prompt.our_prompting import conversion
from api_request.gpt35_turbo_completion import gpt35_turbo_completion
from utils.helper import SpeedLimitTimer


ontology = """Your task is to find the changed domain-slots based on the context and the dialogue between user and system, and find the corresponding value.
The following lists are domain-slots and their possible values.
you don't have to find other changed domain-slots if they are not in the list.

hotel-name: a and b guest house, ashley hotel, el shaddia guest house, etc.
hotel-pricerange: dontcare, cheap, moderate, expensive
hotel-type: hotel, guest house
hotel-parking: dontcare, yes, no
hotel-book_number_of_days: 1, 2, 3, etc.
hotel-book_day: monday, tuesday, etc.
hotel-book_people: 1, 2, 3, etc.
hotel-area: dontcare, centre, east, north, south, west
hotel-stars: dontcare, 0, 1, 2, 3, 4, 5
hotel-internet: dontcare, yes, no

train-destination: london kings cross, cambridge, peterborough, etc.
train-departure: cambridge, stansted airport, etc.
train-day: monday, saturday, etc.
train-book_people: 1, 2, 3, etc.
train-depart_time: 20:24, 12:06, etc.
train-arrive_by_time: 05:51, 20:52, etc.

attraction-name: abbey pool and astroturf pitch, adc theatre, all saints church, castle galleries, etc.
attraction-area: dontcare, centre, east, north, south, west
attraction-type: architecture, boat, church, cinema, college, concert hall, entertainment, hotspot, multiple sports, museum, nightclub, park, special, swimming pool, theatre

restaurant-name: pizza hut city centre, the missing sock, golden wok, cambridge chop house, darrys cookhouse and wine shop, etc.
restaurant-food_type: italian, international, chinese, dontcare, modern european, etc.
restaurant-pricerange: dontcare, cheap, moderate, expensive
restaurant-area: centre, east, north, south, west
restaurant-book_time: 13:30, 17:11, etc.
restaurant-book_day: wednesday, friday, etc.
restaurant-book_people: 1, 2, 3, etc.

taxi-destination: copper kettle, magdalene college, lovell lodge
taxi-departure: royal spice, university arms hotel, da vinci pizzeria
taxi-depart_time: 14:45, 11:15, etc.
taxi-arrive_by_time: 15:30, 12:45, etc.
"""

In [6]:
with open("data/mw24_100p_dev.json", 'r') as f:
    content = json.load(f)

In [32]:
correct = 0
total = 0
domain_len = 0
timer = SpeedLimitTimer(second_per_step=3.1)  # openai limitation 20 queries/min

for idx, data_item in enumerate(content[:]):
    print("********** " + str(idx) + " **********")

    prompt_text = ""
    last_slot_values = {s: v.split(
        '|')[0] for s, v in data_item['last_slot_values'].items()}
    
    prompt_text += f"[context] {conversion(', '.join({f'{slot} = {value}' for slot, value in last_slot_values.items()}))}\n"

    last_sys_utt = data_item['dialog']['sys'][-1]
    if last_sys_utt == 'none':
        last_sys_utt = ''
    prompt_text += f"[system] {last_sys_utt}\n"
    prompt_text += f"[user] {data_item['dialog']['usr'][-1]}\n\n"
    
    prompt_text += """Select the domains of the dialogue.
1. hotel
2. train
3. attraction (architecture, boat, church, cinema, college, concert hall, entertainment, hotspot, multiple sports, museum, nightclub, park, special, swimming pool, theatre)
4. restaurant
5. taxi
"""
#     print(prompt_text)
    
    
#     # gpt35 completion
#     complete_flag = False
#     while not complete_flag:
#         try:
#             completion = gpt35_turbo_completion(prompt_text)
#         except Exception as e:
#             print(e)
#             # throughput too high
#             timer.sleep(10)
#         else:
#             complete_flag = True
#         # limit query speed
#         timer.step()

#     print(f"completion    : {completion}")
#     domain = ["hotel", "train", "attraction", "restaurant", "taxi"]
    
#     predict_domain = set()
#     for domain_idx, i in enumerate(domain):
#         if i in completion or str(domain_idx + 1) in completion:
#             predict_domain.add(i)
#     print(f"predict_domain: {predict_domain}")
    
    label_domain = set([s.split('-', 1)[0] for s,v in data_item['turn_slot_values'].items()])
    print(f"label         : {label_domain}\n")
    
    total += 1
#     if label_domain.issubset(predict_domain): 
#         correct += 1
#         print("correct")
#     else:
#         print("wrong")

    label_domain = set([s.split('-', 1)[0] for s,v in data_item['turn_slot_values'].items()])
    domain_len += len(label_domain)

    print("\n\n")

# print(f"domain accuracy score: {correct / total}")
print(f"domain average len: {domain_len/total}")


********** 0 **********
label         : {'hotel'}




********** 1 **********
label         : {'hotel'}




********** 2 **********
label         : {'hotel'}




********** 3 **********
label         : {'hotel'}




********** 4 **********
label         : {'train'}




********** 5 **********
label         : {'train'}




********** 6 **********
label         : set()




********** 7 **********
label         : {'hotel'}




********** 8 **********
label         : set()




********** 9 **********
label         : {'train'}




********** 10 **********
label         : {'train'}




********** 11 **********
label         : set()




********** 12 **********
label         : {'train'}




********** 13 **********
label         : {'attraction'}




********** 14 **********
label         : {'attraction'}




********** 15 **********
label         : set()




********** 16 **********
label         : set()




********** 17 **********
label         : {'restaurant'}




********** 18 **********
